In [26]:
import numpy as np
import pandas as pd
import datetime
from datetime import date, datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
import time
import catboost as cb
from sklearn.model_selection import KFold
from sklearn.linear_model import BayesianRidge
import workalendar
from workalendar.america import Brazil
from scipy.stats import mode
from sklearn.metrics import log_loss
warnings.filterwarnings('ignore')
np.random.seed(4590)

In [27]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_hist_trans = pd.read_csv('historical_transactions.csv')
df_new_merchant_trans = pd.read_csv('new_merchant_transactions.csv')

In [28]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [29]:
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)
df_hist_trans = reduce_mem_usage(df_hist_trans)
df_new_merchant_trans = reduce_mem_usage(df_new_merchant_trans)

Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)
Mem. usage decreased to 1749.11 Mb (43.7% reduction)
Mem. usage decreased to 114.20 Mb (45.5% reduction)


In [30]:
for df in [df_hist_trans,df_new_merchant_trans]:
    df['category_2'].fillna(1.0,inplace=True)
    df['category_3'].fillna('A',inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 

In [31]:
df_train= pd.get_dummies(df_train, columns=['feature_1', 'feature_2','feature_3'])
df_test = pd.get_dummies(df_test, columns=['feature_1', 'feature_2','feature_3'])

In [32]:
df_hist_trans = pd.get_dummies(df_hist_trans, columns=['category_1','category_2', 'category_3','authorized_flag'])
df_new_merchant_trans = pd.get_dummies(df_new_merchant_trans, columns=['category_1','category_2', 'category_3','authorized_flag'])

In [33]:
df_hist_trans.columns

Index(['card_id', 'city_id', 'installments', 'merchant_category_id',
       'merchant_id', 'month_lag', 'purchase_amount', 'purchase_date',
       'state_id', 'subsector_id', 'category_1_0', 'category_1_1',
       'category_2_1.0', 'category_2_2.0', 'category_2_3.0', 'category_2_4.0',
       'category_2_5.0', 'category_3_A', 'category_3_B', 'category_3_C',
       'authorized_flag_0', 'authorized_flag_1'],
      dtype='object')

In [34]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
def get_other_columns(name,aggs):
    return [name + '_' + k + '_' + 'get_max' for k in aggs]

In [35]:
cal = Brazil()
for yr in [2011,2012,2013,2014,2015,2016,2017]:
    print(yr,cal.holidays(yr))

2011 [(datetime.date(2011, 1, 1), 'New year'), (datetime.date(2011, 4, 21), "Tiradentes' Day"), (datetime.date(2011, 5, 1), 'Labour Day'), (datetime.date(2011, 9, 7), 'Independence Day'), (datetime.date(2011, 10, 12), 'Our Lady of Aparecida'), (datetime.date(2011, 11, 2), "All Souls' Day"), (datetime.date(2011, 11, 15), 'Republic Day'), (datetime.date(2011, 12, 25), 'Christmas Day')]
2012 [(datetime.date(2012, 1, 1), 'New year'), (datetime.date(2012, 4, 21), "Tiradentes' Day"), (datetime.date(2012, 5, 1), 'Labour Day'), (datetime.date(2012, 9, 7), 'Independence Day'), (datetime.date(2012, 10, 12), 'Our Lady of Aparecida'), (datetime.date(2012, 11, 2), "All Souls' Day"), (datetime.date(2012, 11, 15), 'Republic Day'), (datetime.date(2012, 12, 25), 'Christmas Day')]
2013 [(datetime.date(2013, 1, 1), 'New year'), (datetime.date(2013, 4, 21), "Tiradentes' Day"), (datetime.date(2013, 5, 1), 'Labour Day'), (datetime.date(2013, 9, 7), 'Independence Day'), (datetime.date(2013, 10, 12), 'Our Lad

In [36]:
df_hist_trans.shape

(29112361, 22)

In [37]:
for df in [df_hist_trans,df_new_merchant_trans]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
#     df['date'] = df['purchase_date'].dt.date
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    #df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    #df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    df['month_diff'] = ((datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df= pd.get_dummies(df, columns=['weekend'])

In [38]:
df_hist_trans= pd.get_dummies(df_hist_trans, columns=['weekend'])

In [39]:
df_new_merchant_trans= pd.get_dummies(df_new_merchant_trans, columns=['weekend'])

In [55]:
df_new_merchant_trans.columns

Index(['card_id', 'city_id', 'installments', 'merchant_category_id',
       'merchant_id', 'month_lag', 'purchase_amount', 'purchase_date',
       'state_id', 'subsector_id', 'category_1_0', 'category_1_1',
       'category_2_1.0', 'category_2_2.0', 'category_2_3.0', 'category_2_4.0',
       'category_2_5.0', 'category_3_A', 'category_3_B', 'category_3_C',
       'authorized_flag_1', 'year', 'weekofyear', 'month', 'dayofweek', 'hour',
       'month_diff', 'weekend_0', 'weekend_1'],
      dtype='object')

In [41]:
get_max = lambda x:x.sort_values(ascending = False).iloc[0]
get_max.__name__ = "get_max"

In [42]:
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']


aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean']
#aggs['authorized_flag'] = ['sum', 'mean']
#aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

aggs['category_1_0']=[get_max]
aggs['category_1_1']=[get_max]
aggs['category_2_1.0']=[get_max]
aggs['category_2_2.0']=[get_max]
aggs['category_2_3.0']=[get_max]
aggs['category_2_4.0']=[get_max]
aggs['category_2_5.0']=[get_max]
aggs['category_3_A']=[get_max]
aggs['category_3_B']=[get_max]
aggs['category_3_C']=[get_max]
aggs['authorized_flag_0']=[get_max]
aggs['authorized_flag_1']=[get_max]
aggs['weekend_0']=[get_max]
aggs['weekend_1']=[get_max]

In [43]:
agges={'month': ['nunique'],
 'hour': ['nunique'],
 'weekofyear': ['nunique'],
 'dayofweek': ['nunique'],
 'year': ['nunique'],
 'subsector_id': ['nunique'],
 'merchant_id': ['nunique'],
 'merchant_category_id': ['nunique'],
 'purchase_amount': ['sum', 'max', 'min', 'mean', 'var'],
 'installments': ['sum', 'max', 'min', 'mean', 'var'],
 'purchase_date': ['max', 'min'],
 'month_lag': ['max', 'min', 'mean', 'var'],
 'month_diff': ['mean'],
 'card_id': ['size']}

In [76]:
'''for col in ['category_2','category_3']:
    df_hist_trans[col+'_mean'] = df_hist_trans.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean'] 
for col in ['category_2','category_3']:
    df_hist_trans[col+'_install_mean'] = df_hist_trans.groupby([col])['installments'].transform('mean')
    aggs[col+'_install_mean'] = ['mean']  '''

In [44]:
df_hist_trans.columns

Index(['card_id', 'city_id', 'installments', 'merchant_category_id',
       'merchant_id', 'month_lag', 'purchase_amount', 'purchase_date',
       'state_id', 'subsector_id', 'category_1_0', 'category_1_1',
       'category_2_1.0', 'category_2_2.0', 'category_2_3.0', 'category_2_4.0',
       'category_2_5.0', 'category_3_A', 'category_3_B', 'category_3_C',
       'authorized_flag_0', 'authorized_flag_1', 'year', 'weekofyear', 'month',
       'dayofweek', 'hour', 'month_diff', 'weekend_0', 'weekend_1'],
      dtype='object')

In [45]:
agg_other=['category_1_0','category_1_1','category_2_1.0', 'category_2_2.0', 'category_2_3.0', 'category_2_4.0', 'category_2_5.0', 'category_3_A', 'category_3_B', 'category_3_C','authorized_flag_0',
       'authorized_flag_1','weekend_0','weekend_1']

In [46]:
new= get_other_columns('hist',agg_other)
new_columns = get_new_columns('hist',agges)
new_columns=new_columns+new
df_hist_trans_group = df_hist_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['hist_purchase_date_diff'] = (df_hist_trans_group['hist_purchase_date_max'] - df_hist_trans_group['hist_purchase_date_min']).dt.days
df_hist_trans_group['hist_purchase_date_average'] = df_hist_trans_group['hist_purchase_date_diff']/df_hist_trans_group['hist_card_id_size']
df_hist_trans_group['hist_purchase_date_uptonow'] = (datetime.today() - df_hist_trans_group['hist_purchase_date_max']).dt.days
#df_hist_trans_group['hist_purchase_date_mintonow'] = (datetime.today() - df_hist_trans_group['hist_purchase_date_min']).dt.days
df_hist_trans_group['hist_amount_install_percent'] = (df_hist_trans_group['hist_purchase_amount_mean']/df_hist_trans_group['hist_installments_mean'])
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')

#del df_hist_trans_group;
gc.collect()

167

In [58]:
df_new_merchant_trans.drop(columns=['authorized_flag_1'],inplace=True)

In [60]:
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']
    

aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean']
#aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

aggs['category_1_0']=[get_max]
aggs['category_1_1']=[get_max]
aggs['category_2_1.0']=[get_max]
aggs['category_2_2.0']=[get_max]
aggs['category_2_3.0']=[get_max]
aggs['category_2_4.0']=[get_max]
aggs['category_2_5.0']=[get_max]
aggs['category_3_A']=[get_max]
aggs['category_3_B']=[get_max]
aggs['category_3_C']=[get_max]
aggs['weekend_0']=[get_max]
aggs['weekend_1']=[get_max]

In [82]:
'''for col in ['category_2','category_3']:
    df_new_merchant_trans[col+'_mean'] = df_new_merchant_trans.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean']
for col in ['category_2','category_3']:
    df_new_merchant_trans[col+'_install_mean'] = df_new_merchant_trans.groupby([col])['installments'].transform('mean')
    aggs[col+'_install_mean'] = ['mean']  '''

In [61]:
agg_other=['category_1_0','category_1_1','category_2_1.0', 'category_2_2.0', 'category_2_3.0', 'category_2_4.0', 'category_2_5.0', 'category_3_A', 'category_3_B', 'category_3_C','weekend_0','weekend_1']

In [63]:
new= get_other_columns('new_hist',agg_other)
new_columns = get_new_columns('new_hist',agges)
new_columns=new_columns+new
df_hist_trans_group = df_new_merchant_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['new_hist_purchase_date_diff'] = (df_hist_trans_group['new_hist_purchase_date_max'] - df_hist_trans_group['new_hist_purchase_date_min']).dt.days
df_hist_trans_group['new_hist_purchase_date_average'] = df_hist_trans_group['new_hist_purchase_date_diff']/df_hist_trans_group['new_hist_card_id_size']
df_hist_trans_group['new_hist_purchase_date_uptonow'] = (datetime.today() - df_hist_trans_group['new_hist_purchase_date_max']).dt.days
#df_hist_trans_group['new_hist_purchase_date_mintonow'] = (datetime.today() - df_hist_trans_group['new_hist_purchase_date_min']).dt.days
df_hist_trans_group['new_amount_install_percent'] = (df_hist_trans_group['new_hist_purchase_amount_mean']/df_hist_trans_group['new_hist_installments_mean'])
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')

del df_hist_trans_group;
gc.collect()

147

In [64]:
del df_hist_trans;
gc.collect()

del df_new_merchant_trans;
gc.collect()

df_train.head(5)

,first_active_month,card_id,target,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,...,new_hist_category_2_5.0_get_max,new_hist_category_3_A_get_max,new_hist_category_3_B_get_max,new_hist_category_3_C_get_max,new_hist_weekend_0_get_max,new_hist_weekend_1_get_max,new_hist_purchase_date_diff,new_hist_purchase_date_average,new_hist_purchase_date_uptonow,new_amount_install_percent
0,2017-06,C_ID_92a2005557,-0.820312,0,0,0,0,1,0,1,...,0.0,1.0,0.0,0.0,1.0,1.0,54.0,2.347826,269.0,-inf
1,2017-01,C_ID_3d0044924f,0.392822,0,0,0,1,0,1,0,...,0.0,0.0,1.0,0.0,1.0,0.0,56.0,9.333333,299.0,-0.726074
2,2016-08,C_ID_d639edf6cd,0.687988,0,1,0,0,0,0,1,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.000000,269.0,-inf
3,2017-09,C_ID_186d6a6901,0.142456,0,0,0,1,0,0,0,...,0.0,1.0,1.0,0.0,1.0,1.0,41.0,5.857143,280.0,-0.931055
4,2017-11,C_ID_cdbd2c0db2,-0.159790,1,0,0,0,0,0,0,...,0.0,1.0,1.0,1.0,1.0,1.0,57.0,1.583333,269.0,-0.569531


In [65]:
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

In [66]:
df_test.loc[df_test['first_active_month'].isna(),'first_active_month'] = df_test.iloc[11577]['first_active_month']

In [67]:
for df in [df_train,df_test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.today() - df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['hist_last_buy'] = (df['hist_purchase_date_max'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_last_buy'] = (df['new_hist_purchase_date_max'] - df['first_active_month']).dt.days
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']
    df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']
    df['purchase_install_total'] = df['new_hist_installments_sum']+df['hist_installments_sum']
    df['date'] = df['first_active_month'].dt.date
    
     # These are the 8 added features, calculating the no of working days between the first active month and each of the 8 standard Brailian holidays
        
    df['day_diff1'] = df['date'].apply(lambda x: cal.get_working_days_delta(x,cal.holidays(int(x.year))[0][0])) # have to make this less clunky, write a function
    df['day_diff2'] = df['date'].apply(lambda x: cal.get_working_days_delta(x,cal.holidays(int(x.year))[1][0]))
    df['day_diff3'] = df['date'].apply(lambda x: cal.get_working_days_delta(x,cal.holidays(int(x.year))[2][0]))
    df['day_diff4'] = df['date'].apply(lambda x: cal.get_working_days_delta(x,cal.holidays(int(x.year))[3][0]))
    df['day_diff5'] = df['date'].apply(lambda x: cal.get_working_days_delta(x,cal.holidays(int(x.year))[4][0]))
    df['day_diff6'] = df['date'].apply(lambda x: cal.get_working_days_delta(x,cal.holidays(int(x.year))[5][0]))
    df['day_diff7'] = df['date'].apply(lambda x: cal.get_working_days_delta(x,cal.holidays(int(x.year))[6][0]))
    df['day_diff8'] = df['date'].apply(lambda x: cal.get_working_days_delta(x,cal.holidays(int(x.year))[7][0]))
    
    df.drop(['date'],axis=1,inplace=True)

KeyError: 'feature_1'

In [89]:
a = pd.DataFrame()

In [68]:
train = pd.read_csv('train.csv')
train = reduce_mem_usage(train)
test = pd.read_csv('test.csv')
test = reduce_mem_usage(test)

Mem. usage decreased to  4.04 Mb (56.2% reduction)


In [70]:
train['outliers'] = 0
train.loc[train['target'] < -30, 'outliers'] = 1
train['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

In [90]:
for f in ['feature_1','feature_2','feature_3']:
    order_label = train.groupby([f])['outliers'].mean()
    df_train[f] = train[f].map(order_label)
    df_test[f] = test[f].map(order_label)

In [108]:
df_train.columns[:60]

Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'target', 'hist_month_nunique', 'hist_hour_nunique',
       'hist_weekofyear_nunique', 'hist_dayofweek_nunique',
       'hist_year_nunique', 'hist_subsector_id_nunique',
       'hist_merchant_id_nunique', 'hist_merchant_category_id_nunique',
       'hist_purchase_amount_sum', 'hist_purchase_amount_max',
       'hist_purchase_amount_min', 'hist_purchase_amount_mean',
       'hist_purchase_amount_var', 'hist_installments_sum',
       'hist_installments_max', 'hist_installments_min',
       'hist_installments_mean', 'hist_installments_var',
       'hist_purchase_date_max', 'hist_purchase_date_min',
       'hist_month_lag_max', 'hist_month_lag_min', 'hist_month_lag_mean',
       'hist_month_lag_var', 'hist_month_diff_mean',
       'hist_authorized_flag_sum', 'hist_authorized_flag_mean',
       'hist_weekend_sum', 'hist_weekend_mean', 'hist_category_1_sum',
       'hist_category_1_mean', 'hist_card_id_size

In [100]:
final=pd.read_csv('national holidays/card_merchant.csv')

In [101]:
final.head()

,card_id,merchant_id_max,merchant_id_count
0,C_ID_00007093c1,M_ID_9400cf2342,42.0
1,C_ID_0001238066,M_ID_d17aabd756,24.0
2,C_ID_0001506ef0,M_ID_b1fc88154d,13.0
3,C_ID_0001793786,M_ID_923d57de8d,17.0
4,C_ID_000183fdda,M_ID_f9cfe0a43b,13.0


In [110]:
train=pd.merge(df_train,final,how='left',on='card_id')
test=pd.merge(df_test,final,how='left',on='card_id')

In [147]:
test.shape

(123623, 95)

In [155]:
test.shape

(123623, 95)

In [10]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,hist_month_nunique,hist_hour_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,...,day_diff1,day_diff2,day_diff3,day_diff4,day_diff5,day_diff6,day_diff7,day_diff8,merchant_id_max,merchant_id_count
0,2017-06-01,C_ID_92a2005557,0.013145,0.008752,0.011428,-0.8203,9,23,35,7,...,107,28,23,69,93,107,115,142,M_ID_1a81c358a3,32.0
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011385,0.010283,0.3928,12,24,50,7,...,0,79,84,176,200,214,222,249,M_ID_940fb4498f,27.0
2,2016-08-01,C_ID_d639edf6cd,0.010610,0.008752,0.010283,0.6880,10,14,22,7,...,150,72,66,26,50,64,72,100,M_ID_5634fd83e0,28.0
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014166,0.010283,0.1425,6,16,20,7,...,173,94,89,3,27,41,49,76,M_ID_00a6ca8a8a,9.0
4,2017-11-01,C_ID_cdbd2c0db2,0.008058,0.014166,0.010283,-0.1598,4,22,17,7,...,214,135,130,38,14,0,8,35,M_ID_cecefd9589,28.0


In [89]:
merchants=pd.read_csv('merchants.csv')

In [101]:
merchants.head()

,merchant_id_max,mer_merchant_group_id,mer_merchant_category_id,mer_subsector_id,mer_numerical_1,mer_numerical_2,mer_category_1,mer_most_recent_sales_range,mer_most_recent_purchases_range,mer_avg_sales_lag3,...,mer_avg_sales_lag6,mer_avg_purchases_lag6,mer_active_months_lag6,mer_avg_sales_lag12,mer_avg_purchases_lag12,mer_active_months_lag12,mer_category_4,mer_city_id,mer_state_id,mer_category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,0,1,1,-0.40,...,-2.25,18.666667,6,-2.32,13.916667,12,0,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,0,1,1,-0.72,...,-0.74,1.291667,6,-0.57,1.687500,12,0,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,0,1,1,-82.13,...,-82.13,260.000000,2,-82.13,260.000000,2,0,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,1,1,1,1.00,...,1.00,4.666667,6,1.00,3.833333,12,1,-1,-1,1.0
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,1,1,1,1.00,...,1.00,0.361111,6,1.00,0.347222,12,1,-1,-1,1.0


In [76]:
merchants.columns

Index(['merchant_id', 'merchant_group_id', 'merchant_category_id',
       'subsector_id', 'numerical_1', 'numerical_2', 'category_1',
       'most_recent_sales_range', 'most_recent_purchases_range',
       'avg_sales_lag3', 'avg_purchases_lag3', 'active_months_lag3',
       'avg_sales_lag6', 'avg_purchases_lag6', 'active_months_lag6',
       'avg_sales_lag12', 'avg_purchases_lag12', 'active_months_lag12',
       'category_4', 'city_id', 'state_id', 'category_2'],
      dtype='object')

In [90]:
merchants['category_2'].fillna(mode(merchants['category_2']).mode[0], inplace=True)
merchants['avg_sales_lag12'].fillna(mode(merchants['avg_sales_lag12']).mode[0], inplace=True)
merchants['avg_sales_lag6'].fillna(mode(merchants['avg_sales_lag6']).mode[0], inplace=True)
merchants['avg_sales_lag3'].fillna(mode(merchants['avg_sales_lag3']).mode[0], inplace=True)

In [91]:
where_are_inf = np.isinf(merchants['avg_purchases_lag12'])
merchants['avg_purchases_lag12'][where_are_inf] = 0

In [92]:
where_are_inf = np.isinf(merchants['avg_purchases_lag6'])
merchants['avg_purchases_lag6'][where_are_inf] = 0

In [93]:
where_are_inf = np.isinf(merchants['avg_purchases_lag3'])
merchants['avg_purchases_lag3'][where_are_inf] = 0

In [94]:
merchants['category_1'] = merchants['category_1'].map({'Y':1, 'N':0}) 
merchants['most_recent_sales_range'] = merchants['most_recent_sales_range'].map({'A':5, 'B':4, 'C':3, 'D':2, 'E':1}) 
merchants['most_recent_purchases_range'] = merchants['most_recent_purchases_range'].map({'A':5, 'B':4, 'C':3, 'D':2, 'E':1}) 
merchants['category_4'] = merchants['category_4'].map({'Y':1, 'N':0}) 

In [82]:
merchants.to_csv('national holidays/merchants.csv',index=False)

In [95]:
merchants.rename(columns={'merchant_id':'merchant_id_max'},inplace=True)

In [96]:
merchants.columns = ['mer_' + c if c != 'merchant_id_max' else c for c in merchants.columns]

In [97]:
merchants.head()

,merchant_id_max,mer_merchant_group_id,mer_merchant_category_id,mer_subsector_id,mer_numerical_1,mer_numerical_2,mer_category_1,mer_most_recent_sales_range,mer_most_recent_purchases_range,mer_avg_sales_lag3,...,mer_avg_sales_lag6,mer_avg_purchases_lag6,mer_active_months_lag6,mer_avg_sales_lag12,mer_avg_purchases_lag12,mer_active_months_lag12,mer_category_4,mer_city_id,mer_state_id,mer_category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,0,1,1,-0.40,...,-2.25,18.666667,6,-2.32,13.916667,12,0,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,0,1,1,-0.72,...,-0.74,1.291667,6,-0.57,1.687500,12,0,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,0,1,1,-82.13,...,-82.13,260.000000,2,-82.13,260.000000,2,0,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,1,1,1,1.00,...,1.00,4.666667,6,1.00,3.833333,12,1,-1,-1,1.0
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,1,1,1,1.00,...,1.00,0.361111,6,1.00,0.347222,12,1,-1,-1,1.0


In [98]:
merchants=merchants.drop_duplicates('merchant_id_max',keep='first')

In [112]:
train=pd.merge(train,merchants,how='left',on='merchant_id_max')
test=pd.merge(test,merchants,how='left',on='merchant_id_max')

In [221]:
test.shape

(123623, 116)

In [113]:
train.to_csv('national holidays/train_add_done.csv',index=False)
test.to_csv('national holidays/test_add_done.csv',index=False)

In [184]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,hist_month_nunique,hist_hour_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,hist_year_nunique,...,mer_avg_sales_lag6,mer_avg_purchases_lag6,mer_active_months_lag6,mer_avg_sales_lag12,mer_avg_purchases_lag12,mer_active_months_lag12,mer_category_4,mer_city_id,mer_state_id,mer_category_2
0,2017-06-01,C_ID_92a2005557,0.013145,0.008752,0.011428,9,23,35,7,2,...,0.90,0.953233,6,0.93,0.981699,12,1,69,9,1.0
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011385,0.010283,12,24,50,7,2,...,1.10,1.081449,6,1.17,1.132069,12,1,69,9,1.0
2,2016-08-01,C_ID_d639edf6cd,0.010610,0.008752,0.010283,10,14,22,7,2,...,1.01,1.033525,6,1.02,1.052752,12,1,143,5,5.0
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014166,0.010283,6,16,20,7,2,...,8.42,15.855769,6,8.57,16.219780,7,1,-1,-1,1.0
4,2017-11-01,C_ID_cdbd2c0db2,0.008058,0.014166,0.010283,4,22,17,7,2,...,1.04,1.054094,6,1.02,1.055007,12,1,17,22,4.0


In [103]:
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,hist_month_nunique,hist_hour_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,hist_year_nunique,...,mer_avg_sales_lag6,mer_avg_purchases_lag6,mer_active_months_lag6,mer_avg_sales_lag12,mer_avg_purchases_lag12,mer_active_months_lag12,mer_category_4,mer_city_id,mer_state_id,mer_category_2
0,2017-04-01,C_ID_0ab67a22ab,0.010479,0.014166,0.011428,9,17,24,7,1,...,0.82,0.783000,6,0.65,0.584000,12,1,-1,-1,1.0
1,2017-01-01,C_ID_130fd0cbdd,0.010610,0.014166,0.010283,5,18,20,7,2,...,0.93,0.886179,6,0.89,0.832031,12,0,-1,13,4.0
2,2017-08-01,C_ID_b709037bc5,0.013145,0.011385,0.011428,6,7,7,4,2,...,0.90,0.986550,6,0.85,0.968480,12,1,143,5,5.0
3,2017-12-01,C_ID_d27d835a9f,0.010610,0.011385,0.010283,3,9,6,7,2,...,0.96,1.030919,6,0.90,1.000693,12,1,69,9,1.0
4,2015-12-01,C_ID_2b5e3df5c2,0.013145,0.011385,0.011428,12,21,34,7,2,...,0.98,1.024262,6,1.34,1.183017,12,1,277,13,4.0


train比test多target与outliers这两个特征

In [75]:
train=pd.read_csv('national holidays/train_done.csv')
test=pd.read_csv('national holidays/test_done.csv')

In [118]:
train.shape

(201917, 129)

In [114]:
train['mer_active_months_lag3']=train['mer_active_months_lag3']/3
test['mer_active_months_lag3']=test['mer_active_months_lag3']/3

In [115]:
train['mer_active_months_lag6']=train['mer_active_months_lag6']/6
test['mer_active_months_lag6']=test['mer_active_months_lag6']/6

In [116]:
train['mer_active_months_lag12']=train['mer_active_months_lag12']/12
test['mer_active_months_lag12']=test['mer_active_months_lag12']/12

In [69]:
'''cols=['mer_most_recent_sales_range', 'mer_most_recent_purchases_range',
       'mer_avg_sales_lag3', 'mer_avg_purchases_lag3',
        'mer_avg_sales_lag6',
       'mer_avg_purchases_lag6', 
       'mer_avg_sales_lag12', 'mer_avg_purchases_lag12']'''

In [70]:
'''for c in cols:
    min_max_scaler = preprocessing.MinMaxScaler()   ##归一化处理
    train[c] = min_max_scaler.fit_transform(train[c].values.reshape(-1,1)) 
for c in cols:
    min_max_scaler = preprocessing.MinMaxScaler()   ##归一化处理
    test[c] = min_max_scaler.fit_transform(test[c].values.reshape(-1,1)) '''

In [ ]:
train.to_csv('national holidays/train_per.csv',index=False)
test.to_csv('national holidays/test_per.csv',index=False)

# 训练

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import catboost as cb
import xgboost as xgb
import numpy as np
from xgboost import plot_importance
from sklearn.preprocessing import Imputer
from sklearn.model_selection import GridSearchCV

In [9]:
features = [c for c in train.columns if c not in ['card_id', 'first_active_month','target','merchant_id','merchant_id_max','mer_merchant_group_id','mer_merchant_category_id', 'mer_subsector_id','mer_city_id', 'mer_state_id','outliers']]
categorical_feats = [c for c in features if 'feature_' in c]
target = df_train['target']
#del train['target']

In [16]:
folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof_cat = np.zeros(len(train))
predictions_cat = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    model =cb.CatBoostRegressor(iterations=500,learning_rate=0.05,
                            depth=10,
                            eval_metric='RMSE',
                            random_seed = 42,
                            bagging_temperature = 0.2,
                            od_type='Iter',
                            metric_period = 50,
                            l2_leaf_reg=10)
    trn_data =train.iloc[trn_idx][features]
    val_data =train.iloc[val_idx][features]
    tar=target.iloc[trn_idx]
    
    model.fit(trn_data, tar)
    oof_cat[val_idx]= model.predict(val_data)
    
    predictions_cat += model.predict(test[features]) / folds.n_splits

print("CV score: {:<8.5f}".format(mean_squared_error(oof_cat, target)**0.5))

0:	learn: 3.8464236	total: 218ms	remaining: 1m 48s
50:	learn: 3.5947122	total: 10.2s	remaining: 1m 30s
100:	learn: 3.5407738	total: 20s	remaining: 1m 18s
150:	learn: 3.5085665	total: 30.3s	remaining: 1m 10s
200:	learn: 3.4647364	total: 40.3s	remaining: 59.9s
250:	learn: 3.4251515	total: 50.1s	remaining: 49.7s
300:	learn: 3.3870935	total: 1m	remaining: 40.1s
350:	learn: 3.3554662	total: 1m 10s	remaining: 30s
400:	learn: 3.3205170	total: 1m 20s	remaining: 19.9s
450:	learn: 3.2936083	total: 1m 30s	remaining: 9.88s
499:	learn: 3.2734314	total: 1m 40s	remaining: 0us
0:	learn: 3.8660332	total: 221ms	remaining: 1m 50s
50:	learn: 3.6094706	total: 11.3s	remaining: 1m 39s
100:	learn: 3.5395446	total: 22.6s	remaining: 1m 29s
150:	learn: 3.4973139	total: 33s	remaining: 1m 16s
200:	learn: 3.4587908	total: 43.3s	remaining: 1m 4s
250:	learn: 3.4254718	total: 53.7s	remaining: 53.2s
300:	learn: 3.3901672	total: 1m 4s	remaining: 42.4s
350:	learn: 3.3608054	total: 1m 14s	remaining: 31.6s
400:	learn: 3.33

NameError: name 'oof_xgb' is not defined

In [15]:
'''cv_params = {'learning_rate':[0.05,0.1],'iterations':[450,500,550,600]}

optimized_GBM = GridSearchCV(estimator= cb.CatBoostRegressor(depth=10,l2_leaf_reg=10)
                             , param_grid=cv_params, scoring='r2', cv=3, verbose=1)
optimized_GBM.fit(train[features], target)
evalute_result = optimized_GBM.grid_scores_
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))'''

Fitting 3 folds for each of 8 candidates, totalling 24 fits
0:	learn: 3.8702448	total: 253ms	remaining: 1m 53s
1:	learn: 3.8568984	total: 491ms	remaining: 1m 49s
2:	learn: 3.8433801	total: 716ms	remaining: 1m 46s
3:	learn: 3.8310927	total: 926ms	remaining: 1m 43s
4:	learn: 3.8194424	total: 1.13s	remaining: 1m 40s
5:	learn: 3.8089554	total: 1.33s	remaining: 1m 38s
6:	learn: 3.7990882	total: 1.54s	remaining: 1m 37s
7:	learn: 3.7895044	total: 1.74s	remaining: 1m 35s
8:	learn: 3.7797116	total: 1.94s	remaining: 1m 34s
9:	learn: 3.7707096	total: 2.14s	remaining: 1m 34s
10:	learn: 3.7626669	total: 2.33s	remaining: 1m 32s
11:	learn: 3.7552389	total: 2.52s	remaining: 1m 32s
12:	learn: 3.7475299	total: 2.71s	remaining: 1m 31s
13:	learn: 3.7409403	total: 2.92s	remaining: 1m 30s
14:	learn: 3.7347570	total: 3.1s	remaining: 1m 30s
15:	learn: 3.7300209	total: 3.3s	remaining: 1m 29s
16:	learn: 3.7238435	total: 3.57s	remaining: 1m 30s
17:	learn: 3.7189110	total: 3.79s	remaining: 1m 31s
18:	learn: 3.714

158:	learn: 3.5065934	total: 29.3s	remaining: 53.6s
159:	learn: 3.5062236	total: 29.4s	remaining: 53.3s
160:	learn: 3.5059677	total: 29.6s	remaining: 53.1s
161:	learn: 3.5058688	total: 29.8s	remaining: 52.9s
162:	learn: 3.5054013	total: 30s	remaining: 52.8s
163:	learn: 3.5042985	total: 30.2s	remaining: 52.6s
164:	learn: 3.5023020	total: 30.4s	remaining: 52.5s
165:	learn: 3.5020397	total: 30.6s	remaining: 52.3s
166:	learn: 3.5014659	total: 30.7s	remaining: 52.1s
167:	learn: 3.5009907	total: 30.9s	remaining: 51.9s
168:	learn: 3.5005264	total: 31.1s	remaining: 51.7s
169:	learn: 3.5003975	total: 31.3s	remaining: 51.5s
170:	learn: 3.4999242	total: 31.4s	remaining: 51.3s
171:	learn: 3.4991016	total: 31.6s	remaining: 51.1s
172:	learn: 3.4986556	total: 31.8s	remaining: 50.9s
173:	learn: 3.4984071	total: 31.9s	remaining: 50.6s
174:	learn: 3.4979719	total: 32.1s	remaining: 50.4s
175:	learn: 3.4973056	total: 32.3s	remaining: 50.3s
176:	learn: 3.4968612	total: 32.5s	remaining: 50.1s
177:	learn: 3.

317:	learn: 3.4155915	total: 58.3s	remaining: 24.2s
318:	learn: 3.4147334	total: 58.5s	remaining: 24s
319:	learn: 3.4140370	total: 58.6s	remaining: 23.8s
320:	learn: 3.4136951	total: 58.8s	remaining: 23.6s
321:	learn: 3.4136102	total: 59s	remaining: 23.4s
322:	learn: 3.4134761	total: 59.1s	remaining: 23.3s
323:	learn: 3.4128567	total: 59.3s	remaining: 23.1s
324:	learn: 3.4124541	total: 59.5s	remaining: 22.9s
325:	learn: 3.4118193	total: 59.7s	remaining: 22.7s
326:	learn: 3.4110048	total: 60s	remaining: 22.6s
327:	learn: 3.4107280	total: 1m	remaining: 22.4s
328:	learn: 3.4106320	total: 1m	remaining: 22.2s
329:	learn: 3.4102473	total: 1m	remaining: 22s
330:	learn: 3.4100741	total: 1m	remaining: 21.8s
331:	learn: 3.4093720	total: 1m	remaining: 21.6s
332:	learn: 3.4089031	total: 1m 1s	remaining: 21.4s
333:	learn: 3.4085420	total: 1m 1s	remaining: 21.3s
334:	learn: 3.4083802	total: 1m 1s	remaining: 21.1s
335:	learn: 3.4074894	total: 1m 1s	remaining: 20.9s
336:	learn: 3.4072939	total: 1m 1s	

26:	learn: 3.6553127	total: 4.99s	remaining: 1m 18s
27:	learn: 3.6516217	total: 5.17s	remaining: 1m 17s
28:	learn: 3.6484562	total: 5.35s	remaining: 1m 17s
29:	learn: 3.6456527	total: 5.52s	remaining: 1m 17s
30:	learn: 3.6419685	total: 5.7s	remaining: 1m 17s
31:	learn: 3.6386494	total: 5.88s	remaining: 1m 16s
32:	learn: 3.6344593	total: 6.06s	remaining: 1m 16s
33:	learn: 3.6316011	total: 6.23s	remaining: 1m 16s
34:	learn: 3.6289168	total: 6.41s	remaining: 1m 16s
35:	learn: 3.6260399	total: 6.58s	remaining: 1m 15s
36:	learn: 3.6233813	total: 6.76s	remaining: 1m 15s
37:	learn: 3.6213636	total: 6.93s	remaining: 1m 15s
38:	learn: 3.6190601	total: 7.11s	remaining: 1m 14s
39:	learn: 3.6178443	total: 7.27s	remaining: 1m 14s
40:	learn: 3.6145744	total: 7.47s	remaining: 1m 14s
41:	learn: 3.6128727	total: 7.65s	remaining: 1m 14s
42:	learn: 3.6100703	total: 7.84s	remaining: 1m 14s
43:	learn: 3.6080315	total: 8.01s	remaining: 1m 13s
44:	learn: 3.6055889	total: 8.19s	remaining: 1m 13s
45:	learn: 3.

187:	learn: 3.4894759	total: 33.3s	remaining: 46.4s
188:	learn: 3.4887042	total: 33.5s	remaining: 46.2s
189:	learn: 3.4874568	total: 33.7s	remaining: 46.1s
190:	learn: 3.4870274	total: 33.8s	remaining: 45.9s
191:	learn: 3.4868724	total: 34s	remaining: 45.8s
192:	learn: 3.4863235	total: 34.3s	remaining: 45.6s
193:	learn: 3.4862305	total: 34.4s	remaining: 45.5s
194:	learn: 3.4856993	total: 34.6s	remaining: 45.3s
195:	learn: 3.4850901	total: 34.8s	remaining: 45.1s
196:	learn: 3.4849208	total: 35s	remaining: 44.9s
197:	learn: 3.4844649	total: 35.1s	remaining: 44.7s
198:	learn: 3.4836299	total: 35.3s	remaining: 44.5s
199:	learn: 3.4834987	total: 35.5s	remaining: 44.3s
200:	learn: 3.4826512	total: 35.6s	remaining: 44.2s
201:	learn: 3.4821005	total: 35.8s	remaining: 44s
202:	learn: 3.4820041	total: 36s	remaining: 43.8s
203:	learn: 3.4818949	total: 36.1s	remaining: 43.6s
204:	learn: 3.4817518	total: 36.3s	remaining: 43.4s
205:	learn: 3.4813554	total: 36.5s	remaining: 43.2s
206:	learn: 3.480042

...

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 40.1min finished


0:	learn: 3.8552247	total: 235ms	remaining: 1m 57s
1:	learn: 3.8418153	total: 481ms	remaining: 1m 59s
2:	learn: 3.8288409	total: 721ms	remaining: 1m 59s
3:	learn: 3.8177377	total: 971ms	remaining: 2m
4:	learn: 3.8056509	total: 1.21s	remaining: 2m
5:	learn: 3.7947957	total: 1.46s	remaining: 1m 59s
6:	learn: 3.7844936	total: 1.7s	remaining: 1m 59s
7:	learn: 3.7750971	total: 1.96s	remaining: 2m
8:	learn: 3.7663697	total: 2.21s	remaining: 2m
9:	learn: 3.7585816	total: 2.45s	remaining: 1m 59s
10:	learn: 3.7509999	total: 2.69s	remaining: 1m 59s
11:	learn: 3.7440678	total: 2.92s	remaining: 1m 58s
12:	learn: 3.7370792	total: 3.16s	remaining: 1m 58s
13:	learn: 3.7305685	total: 3.4s	remaining: 1m 58s
14:	learn: 3.7241872	total: 3.64s	remaining: 1m 57s
15:	learn: 3.7178518	total: 3.89s	remaining: 1m 57s
16:	learn: 3.7121641	total: 4.13s	remaining: 1m 57s
17:	learn: 3.7066927	total: 4.38s	remaining: 1m 57s
18:	learn: 3.7028719	total: 4.61s	remaining: 1m 56s
19:	learn: 3.6975648	total: 4.86s	remain

158:	learn: 3.5375601	total: 37.6s	remaining: 1m 20s
159:	learn: 3.5373210	total: 37.8s	remaining: 1m 20s
160:	learn: 3.5365086	total: 38.1s	remaining: 1m 20s
161:	learn: 3.5362866	total: 38.3s	remaining: 1m 19s
162:	learn: 3.5361095	total: 38.5s	remaining: 1m 19s
163:	learn: 3.5355261	total: 38.7s	remaining: 1m 19s
164:	learn: 3.5341430	total: 39s	remaining: 1m 19s
165:	learn: 3.5336253	total: 39.2s	remaining: 1m 18s
166:	learn: 3.5332260	total: 39.5s	remaining: 1m 18s
167:	learn: 3.5323799	total: 39.7s	remaining: 1m 18s
168:	learn: 3.5319972	total: 39.9s	remaining: 1m 18s
169:	learn: 3.5310932	total: 40.1s	remaining: 1m 17s
170:	learn: 3.5302419	total: 40.4s	remaining: 1m 17s
171:	learn: 3.5295568	total: 40.6s	remaining: 1m 17s
172:	learn: 3.5292783	total: 40.9s	remaining: 1m 17s
173:	learn: 3.5284513	total: 41.1s	remaining: 1m 17s
174:	learn: 3.5281231	total: 41.4s	remaining: 1m 16s
175:	learn: 3.5280254	total: 41.6s	remaining: 1m 16s
176:	learn: 3.5275747	total: 41.9s	remaining: 1m

316:	learn: 3.4635989	total: 1m 14s	remaining: 43s
317:	learn: 3.4635156	total: 1m 14s	remaining: 42.7s
318:	learn: 3.4627908	total: 1m 14s	remaining: 42.5s
319:	learn: 3.4626410	total: 1m 15s	remaining: 42.3s
320:	learn: 3.4620003	total: 1m 15s	remaining: 42s
321:	learn: 3.4615519	total: 1m 15s	remaining: 41.8s
322:	learn: 3.4614183	total: 1m 15s	remaining: 41.5s
323:	learn: 3.4604615	total: 1m 16s	remaining: 41.3s
324:	learn: 3.4600184	total: 1m 16s	remaining: 41.1s
325:	learn: 3.4595625	total: 1m 16s	remaining: 40.8s
326:	learn: 3.4588287	total: 1m 16s	remaining: 40.6s
327:	learn: 3.4583387	total: 1m 16s	remaining: 40.4s
328:	learn: 3.4580301	total: 1m 17s	remaining: 40.1s
329:	learn: 3.4573020	total: 1m 17s	remaining: 39.9s
330:	learn: 3.4571563	total: 1m 17s	remaining: 39.7s
331:	learn: 3.4563784	total: 1m 17s	remaining: 39.4s
332:	learn: 3.4556818	total: 1m 18s	remaining: 39.2s
333:	learn: 3.4552752	total: 1m 18s	remaining: 39s
334:	learn: 3.4545986	total: 1m 18s	remaining: 38.8s

472:	learn: 3.3952297	total: 1m 51s	remaining: 6.38s
473:	learn: 3.3950950	total: 1m 51s	remaining: 6.14s
474:	learn: 3.3946010	total: 1m 52s	remaining: 5.91s
475:	learn: 3.3944152	total: 1m 52s	remaining: 5.67s
476:	learn: 3.3940912	total: 1m 52s	remaining: 5.43s
477:	learn: 3.3938703	total: 1m 52s	remaining: 5.2s
478:	learn: 3.3934966	total: 1m 53s	remaining: 4.96s
479:	learn: 3.3931447	total: 1m 53s	remaining: 4.72s
480:	learn: 3.3930347	total: 1m 53s	remaining: 4.49s
481:	learn: 3.3926802	total: 1m 53s	remaining: 4.25s
482:	learn: 3.3925967	total: 1m 54s	remaining: 4.01s
483:	learn: 3.3917274	total: 1m 54s	remaining: 3.78s
484:	learn: 3.3915567	total: 1m 54s	remaining: 3.54s
485:	learn: 3.3907881	total: 1m 54s	remaining: 3.31s
486:	learn: 3.3904577	total: 1m 55s	remaining: 3.07s
487:	learn: 3.3903358	total: 1m 55s	remaining: 2.83s
488:	learn: 3.3900032	total: 1m 55s	remaining: 2.6s
489:	learn: 3.3891339	total: 1m 55s	remaining: 2.37s
490:	learn: 3.3882586	total: 1m 56s	remaining: 2

In [147]:
folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof_xgb = np.zeros(len(train))
predictions_xgb = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    model = xgb.XGBRegressor(max_depth=5, learning_rate=0.1, n_estimators=100,min_child_weight=6, gamma=0.1,colsample_bytree=0.8, subsample=0.9,reg_alpha=3, reg_lambda=3, silent=False, objective='reg:linear')
    trn_data =train.iloc[trn_idx][features]
    val_data =train.iloc[val_idx][features]
    tar=target.iloc[trn_idx]
    
    model.fit(trn_data, tar)
    oof_xgb[val_idx]= model.predict(val_data)
    
    predictions_xgb += model.predict(test[features]) / folds.n_splits

print("CV score: {:<8.5f}".format(mean_squared_error(oof_xgb, target)**0.5))

[19:04:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[19:04:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[19:04:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[19:04:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[19:04:54] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[19:04:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[19:04:56] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[19:05:38] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[19:05:39] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[19:05:39] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[19:05:40] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[19:05:41] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[19:05:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=5
[19:05:43] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[19:06:27] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[19:06:27] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[19:06:28] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[19:06:29] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[19:06:30] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[19:06:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[19:06:32] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[19:07:15] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[19:07:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[19:07:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[19:07:17] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=5
[19:07:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[19:07:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[19:07:20] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[19:08:04] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[19:08:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[19:08:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[19:08:07] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[19:08:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[19:08:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[19:08:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[19:08:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[19:08:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[19:08:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[19:08:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[19:08:54] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[19:08:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[19:08:56] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[19:09:40] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[19:09:41] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[19:09:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[19:09:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[19:09:43] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[19:09:44] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[19:09:45] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[19:10:26] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[19:10:27] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=5
[19:10:28] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=5
[19:10:29] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=5
[19:10:30] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[19:10:30] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=5
[19:10:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[19:11:15] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[19:11:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[19:11:17] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[19:11:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[19:11:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=5
[19:11:20] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[19:11:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[19:12:01] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=5
[19:12:02] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[19:12:03] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[19:12:04] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[19:12:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=5
CV score: 3.66140 


In [18]:
from sklearn.model_selection import RepeatedKFold
folds = RepeatedKFold(n_splits=5, n_repeats=2, random_state=4520)

oof_lgb_xgb = np.zeros(len(train))
predictions_lgb_xgb = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    model = xgb.XGBRegressor(max_depth=5, learning_rate=0.1, n_estimators=100,min_child_weight=6, gamma=0.1,colsample_bytree=0.8, subsample=0.9,reg_alpha=3, reg_lambda=3, silent=False, objective='reg:linear')
    trn_data =train.iloc[trn_idx][features]
    val_data =train.iloc[val_idx][features]
    tar=target.iloc[trn_idx]
    
    model.fit(trn_data, tar)
    oof_lgb_xgb[val_idx]= model.predict(val_data)
    
    predictions_lgb_xgb += model.predict(test[features]) / (5 * 2)

print("CV score: {:<8.5f}".format(mean_squared_error(oof_lgb_xgb, target)**0.5))

[16:22:00] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[16:22:01] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[16:22:02] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[16:22:03] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[16:22:04] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[16:22:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[16:22:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[16:22:48] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[16:22:49] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[16:22:50] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[16:22:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[16:22:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[16:22:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[16:22:54] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[16:23:38] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[16:23:39] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[16:23:40] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[16:23:41] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[16:23:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[16:23:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[16:23:43] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[16:24:26] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=5
[16:24:27] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[16:24:28] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[16:24:29] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[16:24:30] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[16:24:30] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[16:24:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[16:25:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[16:25:17] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[16:25:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[16:25:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[16:25:20] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[16:25:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[16:25:22] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

...

[16:33:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[16:33:24] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[16:33:25] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=5
[16:33:26] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[16:33:27] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[16:33:28] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[16:33:28] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[16:34:14] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[16:34:15] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[16:34:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[16:34:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[16:34:17] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[16:34:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[16:34:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[16:35:03] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=5
[16:35:03] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=5
[16:35:04] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[16:35:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[16:35:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[16:35:07] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=5
[16:35:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[16:35:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[16:35:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[16:35:54] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[16:35:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[16:35:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[16:35:56] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[16:35:57] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[16:36:38] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[16:36:38] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[16:36:39] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[16:36:40] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[16:36:41] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=5
[16:36:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=5
[16:36:43] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

In [132]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.005,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "nthread": 4,
         "verbosity": -1}

In [133]:
folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(train))
predictions = np.zeros(len(test))
start = time.time()
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx][features], label=target.iloc[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
    oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(mean_squared_error(oof, target)**0.5))

fold n°0
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.72544	valid_1's rmse: 3.78851
[200]	training's rmse: 3.65889	valid_1's rmse: 3.74436
[300]	training's rmse: 3.61555	valid_1's rmse: 3.72197
[400]	training's rmse: 3.58253	valid_1's rmse: 3.70923
[500]	training's rmse: 3.55613	valid_1's rmse: 3.70123
[600]	training's rmse: 3.53445	valid_1's rmse: 3.69583
[700]	training's rmse: 3.51445	valid_1's rmse: 3.69097
[800]	training's rmse: 3.49757	valid_1's rmse: 3.68745
[900]	training's rmse: 3.48199	valid_1's rmse: 3.68458
[1000]	training's rmse: 3.46719	valid_1's rmse: 3.68253
[1100]	training's rmse: 3.45409	valid_1's rmse: 3.6808
[1200]	training's rmse: 3.44183	valid_1's rmse: 3.67954
[1300]	training's rmse: 3.43007	valid_1's rmse: 3.67834
[1400]	training's rmse: 3.41903	valid_1's rmse: 3.67755
[1500]	training's rmse: 3.40859	valid_1's rmse: 3.67667
[1600]	training's rmse: 3.39831	valid_1's rmse: 3.67616
[1700]	training's rmse: 3.38824	valid_1's 

Early stopping, best iteration is:
[1874]	training's rmse: 3.38283	valid_1's rmse: 3.61428
CV score: 3.65791 


In [135]:
lgbparam = {'num_leaves': 31,
            'boosting_type': 'rf',
             'min_data_in_leaf': 30, 
             'objective':'regression',
             'max_depth': -1,
             'learning_rate': 0.01,
             "min_child_samples": 20,
             "boosting": "gbdt",
             "feature_fraction": 0.9,
             "bagging_freq": 1,
             "bagging_fraction": 0.9 ,
             "bagging_seed": 11,
             "metric": 'rmse',
             "lambda_l1": 0.1,
             "verbosity": -1,
             "nthread": 4,
             "random_state": 4590}

In [136]:
from sklearn.model_selection import RepeatedKFold
folds = RepeatedKFold(n_splits=5, n_repeats=2, random_state=4520)

oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros(len(test))
start = time.time()
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx][features], label=target.iloc[val_idx])

    num_round = 11000
    clf = lgb.train(lgbparam, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
    oof_lgb[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions_lgb += clf.predict(test[features], num_iteration=clf.best_iteration) / (5 * 2)

print("CV score: {:<8.5f}".format(mean_squared_error(oof_lgb, target)**0.5))

fold n°0
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 3.66066	valid_1's rmse: 3.73173
[200]	training's rmse: 3.58079	valid_1's rmse: 3.70616
[300]	training's rmse: 3.53002	valid_1's rmse: 3.69407
[400]	training's rmse: 3.49191	valid_1's rmse: 3.68704
[500]	training's rmse: 3.4621	valid_1's rmse: 3.68238
[600]	training's rmse: 3.43548	valid_1's rmse: 3.67988
[700]	training's rmse: 3.41323	valid_1's rmse: 3.67847
[800]	training's rmse: 3.39244	valid_1's rmse: 3.6766
[900]	training's rmse: 3.37339	valid_1's rmse: 3.67573
[1000]	training's rmse: 3.35474	valid_1's rmse: 3.67489
[1100]	training's rmse: 3.33769	valid_1's rmse: 3.67427
[1200]	training's rmse: 3.32084	valid_1's rmse: 3.67405
Early stopping, best iteration is:
[1191]	training's rmse: 3.32234	valid_1's rmse: 3.67399
fold n°1
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 3.69509	valid_1's rmse: 3.59293
[200]	training's rmse: 3.61584	valid_1's rmse: 3.

In [137]:
train_stack = np.vstack([oof_cat,oof_lgb_xgb,oof,oof_lgb]).transpose()
test_stack = np.vstack([predictions_cat,predictions_lgb_xgb,predictions,predictions_lgb]).transpose()

folds = RepeatedKFold(n_splits=5,n_repeats=1,random_state=4520)
oof_stack = np.zeros(train_stack.shape[0])
predictions_stack = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_stack, target)):
    print("fold n°{}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], target.iloc[trn_idx].values
    val_data, val_y = train_stack[val_idx], target.iloc[val_idx].values

    print("-" * 10 + "Stacking " + str(fold_) + "-" * 10)
#     cb_model = CatBoostRegressor(iterations=3000, learning_rate=0.1, depth=8, l2_leaf_reg=20, bootstrap_type='Bernoulli',  eval_metric='RMSE', metric_period=50, od_type='Iter', od_wait=45, random_seed=17, allow_writing_files=False)
#     cb_model.fit(trn_data, trn_y, eval_set=(val_data, val_y), cat_features=[], use_best_model=True, verbose=True)
    clf = BayesianRidge()
    clf.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf.predict(val_data)
    predictions_stack += clf.predict(test_stack) / 5


np.sqrt(mean_squared_error(target.values, oof_stack))

fold n°0
----------Stacking 0----------
fold n°1
----------Stacking 1----------
fold n°2
----------Stacking 2----------
fold n°3
----------Stacking 3----------
fold n°4
----------Stacking 4----------


3.653658162145467

In [ ]:
3.6557836317003827  3.6551030677541134

In [138]:
predictions_stack.shape

(123623,)

In [139]:
df_test.shape

(123623, 104)

In [142]:
sample_submission.shape

(123623, 2)

In [140]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['target'] = predictions_stack
sample_submission.to_csv('national holidays/mer_month_per.csv', index=False)